In [14]:

from pandas._libs.parsers import defaultdict
from algorithm.order_plan.multi_machines.object.Operation import Operation
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
from datetime import datetime, timedelta
import random
import copy

from algorithm.order_plan.multi_machines.object.order_multi_feature import OrderMultiFeature
from algorithm.order_plan.multi_machines.object.static_parameter import StaticParameters


@dataclass
class JobSchedule:
    """单个合同的调度方案"""
    order_no: str
    operation: Operation
    machine_id: int  # 机组ID
    start_time: float  # 相对时间（小时）
    end_time: float
    weight: float


@dataclass
class ProductionPlan:
    """完整的生产方案"""
    schedules: List[JobSchedule] = field(default_factory=list)
    is_feasible: bool = True
    total_delay: float = 0.0  # 总拖期
    total_cost: float = 0.0  # 总生产成本
    fitness: float = float('inf')  # 适应度值（越小越好）

    def get_schedules_by_operation(self, operation: Operation) -> List[JobSchedule]:
        """获取指定工序的所有调度"""
        return [s for s in self.schedules if s.operation == operation]


@dataclass
class StockState:
    """库存状态追踪"""
    operation: Operation
    time: float
    stock_level: float
    event_type: str  # 'produce' or 'consume'
    order_no: str


class SetupCostCalculator:
    """搭接费用计算器"""

    def __init__(self, orders: List[OrderMultiFeature]):
        self.orders = orders
        self.order_dict = {o.order_no: o for o in orders}
        # 预计算所有合同对的搭接费用
        self.setup_costs = self._precalculate_setup_costs()

    def _precalculate_setup_costs(self) -> Dict[Tuple[Operation, str, str], float]:
        """预计算搭接费用矩阵"""
        costs = {}

        for op in Operation:
            # 不同工序的费用系数
            if op == Operation.HR:
                wide_to_narrow_cost = 10.0
                narrow_to_wide_cost = 50.0
            elif op == Operation.AZ:
                wide_to_narrow_cost = 15.0
                narrow_to_wide_cost = 60.0
            else:  # CA
                wide_to_narrow_cost = 20.0
                narrow_to_wide_cost = 70.0

            for i, order_i in enumerate(self.orders):
                for j, order_j in enumerate(self.orders):
                    if i == j:
                        costs[(op, order_i.order_no, order_j.order_no)] = 0.0
                    else:
                        # 从宽到窄：费用低
                        if order_i.order_width >= order_j.order_width:
                            cost = wide_to_narrow_cost * abs(order_i.order_width - order_j.order_width)
                        else:
                            # 从窄到宽：费用高
                            cost = narrow_to_wide_cost * abs(order_i.order_width - order_j.order_width)

                        costs[(op, order_i.order_no, order_j.order_no)] = cost

        return costs

    def get_cost(self, operation: Operation, from_order: str, to_order: str) -> float:
        """获取两个合同在某工序的搭接费用"""
        return self.setup_costs.get((operation, from_order, to_order), 0.0)


class Chromosome:
    """染色体：合同优先级序列"""

    def __init__(self, priority_sequence: List[int]):
        """
        priority_sequence: 合同索引的优先级序列，例如[2,0,1]表示第3个合同优先级最高
        """
        self.genes = priority_sequence
        self.production_plan: Optional[ProductionPlan] = None
        self.fitness: float = float('inf')

    def __len__(self):
        return len(self.genes)

    def copy(self):
        """深拷贝染色体"""
        new_chrome = Chromosome(self.genes.copy())
        if self.production_plan:
            new_chrome.production_plan = copy.deepcopy(self.production_plan)
        new_chrome.fitness = self.fitness
        return new_chrome


class ScheduleDecoder:
    """调度解码器：将染色体解码为可行的生产方案"""

    def __init__(self, orders: List[OrderMultiFeature], static_params: StaticParameters,
                 setup_calculator: SetupCostCalculator):
        self.orders = orders
        self.params = static_params
        self.setup_calculator = setup_calculator

    def decode(self, chromosome: Chromosome) -> ProductionPlan:
        """解码染色体，生成生产方案（逆向调度）"""
        plan = ProductionPlan()

        # 按优先级排序合同
        sorted_order_indices = chromosome.genes

        # 机组空闲时间追踪 {(operation, machine_id): available_time}
        machine_available_time = {}
        for op in Operation:
            for m_id in range(len(self.params.speed[op])):
                machine_available_time[(op, m_id)] = 0.0

        # 每个合同在各工序的调度结果 {order_no: {operation: JobSchedule}}
        order_schedules: Dict[str, Dict[Operation, JobSchedule]] = {}

        # 库存变化事件列表
        stock_events: List[StockState] = []

        # 按优先级逆向调度每个合同
        for order_idx in sorted_order_indices:
            order = self.orders[order_idx]
            schedules_for_order = {}

            # 逆向调度：从CA开始往前推
            for i in range(len(self.params.operation_sequence) - 1, -1, -1):
                operation = self.params.operation_sequence[i]

                # 选择最快的空闲机组
                best_machine_id = self._select_best_machine(operation, machine_available_time)
                processing_time = order.order_wt / self.params.speed[operation][best_machine_id]

                if i == len(self.params.operation_sequence) - 1:
                    # 最后一道工序：尽可能晚开始，但要满足交货期
                    delivery_time_hours = (order.delivery_date - order.plan_start_date).total_seconds() / 3600
                    trans_time = self.params.transmission_time[operation]
                    latest_end_time = delivery_time_hours - trans_time

                    # 考虑机组可用时间
                    earliest_start = machine_available_time[(operation, best_machine_id)]
                    desired_end_time = max(earliest_start + processing_time, latest_end_time)
                    start_time = desired_end_time - processing_time

                    # 如果机组不够早，只能从机组可用时间开始
                    if start_time < earliest_start:
                        start_time = earliest_start

                    end_time = start_time + processing_time
                else:
                    # 中间工序：需要在下游工序开始前完成并转运
                    downstream_operation = self.params.operation_sequence[i + 1]
                    downstream_schedule = schedules_for_order[downstream_operation]

                    # 必须在下游开始前完成转运
                    required_end_time = downstream_schedule.start_time - self.params.transmission_time[operation]

                    # 考虑机组可用时间
                    earliest_start = machine_available_time[(operation, best_machine_id)]
                    desired_start = required_end_time - processing_time

                    start_time = max(earliest_start, desired_start)
                    end_time = start_time + processing_time

                # 创建调度
                schedule = JobSchedule(
                    order_no=order.order_no,
                    operation=operation,
                    machine_id=best_machine_id,
                    start_time=start_time,
                    end_time=end_time,
                    weight=order.order_wt
                )

                schedules_for_order[operation] = schedule
                machine_available_time[(operation, best_machine_id)] = end_time

                # 记录库存变化事件
                if i < len(self.params.operation_sequence) - 1:
                    # 完成当前工序：库存增加
                    stock_events.append(StockState(
                        operation=operation,
                        time=end_time,
                        stock_level=order.order_wt,
                        event_type='produce',
                        order_no=order.order_no
                    ))

                    # 开始下游工序：库存减少
                    downstream_schedule = schedules_for_order[self.params.operation_sequence[i + 1]]
                    stock_events.append(StockState(
                        operation=operation,
                        time=downstream_schedule.start_time,
                        stock_level=-order.order_wt,
                        event_type='consume',
                        order_no=order.order_no
                    ))

            order_schedules[order.order_no] = schedules_for_order

            # 将该合同的所有调度加入方案
            for schedule in schedules_for_order.values():
                plan.schedules.append(schedule)

        # 检查库存约束
        plan.is_feasible = self._check_stock_constraints(stock_events)

        if plan.is_feasible:
            # 计算拖期
            plan.total_delay = self._calculate_delay(order_schedules)

            # 计算生产成本（搭接费用）
            plan.total_cost = self._calculate_setup_cost(order_schedules)

            # 计算适应度（权重可调）
            delay_weight = 1000.0  # 拖期权重
            cost_weight = 1.0      # 成本权重
            plan.fitness = delay_weight * max(0, plan.total_delay) + cost_weight * plan.total_cost
        else:
            plan.fitness = float('inf')  # 不可行方案

        return plan

    def _select_best_machine(self, operation: Operation,
                             machine_available_time: Dict[Tuple[Operation, int], float]) -> int:
        """选择最快的空闲机组"""
        speeds = self.params.speed[operation]
        best_machine = 0
        earliest_time = float('inf')
        fastest_speed = 0

        for m_id, speed in enumerate(speeds):
            available_time = machine_available_time[(operation, m_id)]
            # 优先选择更快的机组，如果速度相同则选择更早空闲的
            if speed > fastest_speed or (speed == fastest_speed and available_time < earliest_time):
                fastest_speed = speed
                earliest_time = available_time
                best_machine = m_id

        return best_machine

    def _check_stock_constraints(self, stock_events: List[StockState]) -> bool:
        """检查库存约束是否满足"""
        # 按时间排序事件
        stock_events.sort(key=lambda x: x.time)

        # 追踪每个产线的库存
        current_stock = {op: 0.0 for op in Operation}

        for event in stock_events:
            current_stock[event.operation] += event.stock_level

            # 检查是否违反约束
            limits = self.params.stock_limit[event.operation]
            if current_stock[event.operation] < limits[0] or current_stock[event.operation] > limits[1]:
                return False

        return True

    def _calculate_delay(self, order_schedules: Dict[str, Dict[Operation, JobSchedule]]) -> float:
        """计算总拖期"""
        total_delay = 0.0

        for order_no, schedules in order_schedules.items():
            order = next(o for o in self.orders if o.order_no == order_no)

            # 最后一道工序
            last_operation = self.params.operation_sequence[-1]
            last_schedule = schedules[last_operation]

            # 实际完成时间 = 加工结束时间 + 转运时间
            actual_finish_time_hours = last_schedule.end_time + self.params.transmission_time[last_operation]
            actual_finish_date = order.plan_start_date + timedelta(hours=actual_finish_time_hours)

            # 拖期 = 实际完成时间 - 交货期
            delay_hours = (actual_finish_date - order.delivery_date).total_seconds() / 3600

            if delay_hours > 0:
                total_delay += delay_hours

        return total_delay

    def _calculate_setup_cost(self, order_schedules: Dict[str, Dict[Operation, JobSchedule]]) -> float:
        """计算总搭接费用"""
        total_cost = 0.0

        # 对每个工序，找出在同一机组上连续生产的合同对
        for operation in Operation:
            # 按机组分组
            machine_schedules: Dict[int, List[JobSchedule]] = {}

            for schedules in order_schedules.values():
                schedule = schedules[operation]
                if schedule.machine_id not in machine_schedules:
                    machine_schedules[schedule.machine_id] = []
                machine_schedules[schedule.machine_id].append(schedule)

            # 对每个机组，按时间排序并计算搭接费用
            for machine_id, schedules in machine_schedules.items():
                schedules.sort(key=lambda x: x.start_time)

                for i in range(len(schedules) - 1):
                    from_order = schedules[i].order_no
                    to_order = schedules[i + 1].order_no
                    cost = self.setup_calculator.get_cost(operation, from_order, to_order)
                    total_cost += cost

        return total_cost


class GeneticAlgorithm:
    """遗传算法主类"""

    def __init__(self, orders: List[OrderMultiFeature], static_params: StaticParameters,
                 population_size: int = 100, generations: int = 200,
                 crossover_rate: float = 0.8, mutation_rate: float = 0.2):
        self.orders = orders
        self.params = static_params
        self.population_size = population_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate

        self.setup_calculator = SetupCostCalculator(orders)
        self.decoder = ScheduleDecoder(orders, static_params, self.setup_calculator)

        self.population: List[Chromosome] = []
        self.best_chromosome: Optional[Chromosome] = None
        self.best_fitness_history: List[float] = []

    def initialize_population(self):
        """初始化种群"""
        n_orders = len(self.orders)

        for i in range(self.population_size):
            # 使用多种初始化策略
            strategy = i % 5

            if strategy == 0:
                # 按交货期早的优先
                indices = sorted(range(n_orders), key=lambda x: self.orders[x].delivery_date)
            elif strategy == 1:
                # 按交货期晚的优先
                indices = sorted(range(n_orders), key=lambda x: self.orders[x].delivery_date, reverse=True)
            elif strategy == 2:
                # 按计划开始日期早的优先（先来先服务）
                indices = sorted(range(n_orders), key=lambda x: self.orders[x].plan_start_date)
            elif strategy == 3:
                # 按重量大的优先（大订单优先）
                indices = sorted(range(n_orders), key=lambda x: self.orders[x].order_wt, reverse=True)
            else:
                # 随机初始化
                indices = list(range(n_orders))
                random.shuffle(indices)

            chromosome = Chromosome(indices)
            self.population.append(chromosome)

    def evaluate_population(self):
        """评估种群适应度"""
        for chromosome in self.population:
            plan = self.decoder.decode(chromosome)
            chromosome.production_plan = plan
            chromosome.fitness = plan.fitness

    def select_parents(self) -> Tuple[Chromosome, Chromosome]:
        """锦标赛选择"""
        tournament_size = 5

        def tournament():
            competitors = random.sample(self.population, tournament_size)
            return min(competitors, key=lambda x: x.fitness)

        parent1 = tournament()
        parent2 = tournament()
        return parent1, parent2

    def crossover(self, parent1: Chromosome, parent2: Chromosome) -> Tuple[Chromosome, Chromosome]:
        """顺序交叉（OX）"""
        if random.random() > self.crossover_rate:
            return parent1.copy(), parent2.copy()

        size = len(parent1)

        # 随机选择交叉区间
        start, end = sorted(random.sample(range(size), 2))

        # 创建子代1
        child1_genes = [-1] * size
        child1_genes[start:end] = parent1.genes[start:end]

        # 从parent2填充剩余基因
        remaining = [g for g in parent2.genes if g not in child1_genes]
        idx = 0
        for i in range(size):
            if child1_genes[i] == -1:
                child1_genes[i] = remaining[idx]
                idx += 1

        # 创建子代2
        child2_genes = [-1] * size
        child2_genes[start:end] = parent2.genes[start:end]

        remaining = [g for g in parent1.genes if g not in child2_genes]
        idx = 0
        for i in range(size):
            if child2_genes[i] == -1:
                child2_genes[i] = remaining[idx]
                idx += 1

        return Chromosome(child1_genes), Chromosome(child2_genes)

    def mutate(self, chromosome: Chromosome):
        """交换变异"""
        if random.random() < self.mutation_rate:
            idx1, idx2 = random.sample(range(len(chromosome)), 2)
            chromosome.genes[idx1], chromosome.genes[idx2] = chromosome.genes[idx2], chromosome.genes[idx1]

    def evolve(self):
        """进化一代"""
        new_population = []

        # 精英保留
        elite_size = max(1, self.population_size // 10)
        elites = sorted(self.population, key=lambda x: x.fitness)[:elite_size]
        new_population.extend([e.copy() for e in elites])

        # 生成新个体
        while len(new_population) < self.population_size:
            parent1, parent2 = self.select_parents()
            child1, child2 = self.crossover(parent1, parent2)

            self.mutate(child1)
            self.mutate(child2)

            new_population.append(child1)
            if len(new_population) < self.population_size:
                new_population.append(child2)

        self.population = new_population[:self.population_size]

    def run(self) -> Chromosome:
        """运行遗传算法"""
        print(f"初始化种群（大小={self.population_size}）...")
        self.initialize_population()

        print(f"开始进化（代数={self.generations}）...")
        for generation in range(self.generations):
            # 评估
            self.evaluate_population()

            # 记录最优解
            current_best = min(self.population, key=lambda x: x.fitness)
            if self.best_chromosome is None or current_best.fitness < self.best_chromosome.fitness:
                self.best_chromosome = current_best.copy()

            self.best_fitness_history.append(self.best_chromosome.fitness)

            # 输出进度
            if (generation + 1) % 20 == 0 or generation == 0:
                print(f"代数 {generation + 1}/{self.generations}, "
                      f"最优适应度: {self.best_chromosome.fitness:.2f}, "
                      f"拖期: {self.best_chromosome.production_plan.total_delay:.2f}h, "
                      f"成本: {self.best_chromosome.production_plan.total_cost:.2f}")

            # 进化
            if generation < self.generations - 1:
                self.evolve()

        print("\n优化完成!")
        return self.best_chromosome


# 使用示例
if __name__ == "__main__":
    # 创建测试数据
    base_date = datetime(2024, 11, 1)

    orders = [
        OrderMultiFeature(
            order_no=f"ORDER_{i:03d}",
            delivery_date=base_date + timedelta(days=random.randint(2, 4)),
            plan_start_date=base_date,
            order_wt=random.uniform(10, 50),
            order_width=random.uniform(800, 2000),
            order_thick=random.uniform(2, 10)
        )
        for i in range(20)
    ]

    static_params = StaticParameters()

    # 运行遗传算法
    ga = GeneticAlgorithm(
        orders=orders,
        static_params=static_params,
        population_size=100,
        generations=200,
        crossover_rate=0.8,
        mutation_rate=0.2
    )

    best_solution = ga.run()

    print(f"\n最优方案:")
    print(f"优先级序列: {best_solution.genes}")
    print(f"总拖期: {best_solution.production_plan.total_delay:.2f} 小时")
    print(f"总成本: {best_solution.production_plan.total_cost:.2f}")
    print(f"适应度: {best_solution.fitness:.2f}")
    print(f"可行性: {best_solution.production_plan.is_feasible}")

    setup_calculator = SetupCostCalculator(orders)
    decoder = ScheduleDecoder(orders, static_params, setup_calculator)
    result = decoder.decode(best_solution)

    # 按合同和工序分组
    grouped_schedules = defaultdict(dict)

    for schedule in result.schedules:
        grouped_schedules[schedule.order_no][schedule.operation] = schedule

    # 遍历并打印每个合同的工序信息
    for order_no, operations in grouped_schedules.items():

        print(f"合同编号: {order_no}")
        for operation, schedule in operations.items():
            res = ""
            res += f"  工序: {schedule.operation.value}"
            res += f",机器ID: {schedule.machine_id}"
            res += f",开始时间: {round(schedule.start_time,2)}"
            res += f",结束时间: {round(schedule.end_time,2)}"
            res += f",生产时间: {round(schedule.end_time - schedule.start_time,2)} 小时"
            print(res)


初始化种群（大小=100）...
开始进化（代数=200）...
代数 1/200, 最优适应度: 1026694.65, 拖期: 527.00h, 成本: 499691.14
代数 20/200, 最优适应度: 770035.88, 拖期: 349.81h, 成本: 420226.20
代数 40/200, 最优适应度: 592435.01, 拖期: 298.12h, 成本: 294318.71
代数 60/200, 最优适应度: 590605.54, 拖期: 296.29h, 成本: 294318.71
代数 80/200, 最优适应度: 530658.14, 拖期: 266.14h, 成本: 264518.86
代数 100/200, 最优适应度: 530658.14, 拖期: 266.14h, 成本: 264518.86
代数 120/200, 最优适应度: 530658.14, 拖期: 266.14h, 成本: 264518.86
代数 140/200, 最优适应度: 530658.14, 拖期: 266.14h, 成本: 264518.86
代数 160/200, 最优适应度: 530658.14, 拖期: 266.14h, 成本: 264518.86
代数 180/200, 最优适应度: 530658.14, 拖期: 266.14h, 成本: 264518.86
代数 200/200, 最优适应度: 530658.14, 拖期: 266.14h, 成本: 264518.86

优化完成!

最优方案:
优先级序列: [9, 12, 16, 8, 1, 19, 10, 17, 13, 2, 5, 15, 7, 0, 18, 6, 4, 11, 14, 3]
总拖期: 266.14 小时
总成本: 264518.86
适应度: 530658.14
可行性: True
合同编号: ORDER_009
  工序: CA,机器ID: 1,开始时间: 46.08,结束时间: 47.0,生产时间: 0.92 小时
  工序: AZ,机器ID: 1,开始时间: 41.22,结束时间: 42.08,生产时间: 0.86 小时
  工序: HR,机器ID: 0,开始时间: 38.53,结束时间: 39.22,生产时间: 0.69 小时
合同编号: ORDER_012
  